In [1]:
from datahelper import CreateDataset
from PIL import Image
import os
import torchvision

In [2]:


dataset = CreateDataset('annotation_dict.pkl', transform = torchvision.transforms.ToTensor())



In [10]:
dataset[258130]['annotation']

{'face_detected': True,
 'facebox': None,
 'landmarks': {'MIDPOINT_BETWEEN_EYES': {'x': 430,
   'y': 878,
   'z': -49.71061325073242},
  'NOSE_BOTTOM_LEFT': {'x': 328, 'y': 1245, 'z': -10.092453002929688},
  'LEFT_EYEBROW_UPPER_MIDPOINT': {'x': 105, 'y': 736, 'z': -40.40224075317383},
  'RIGHT_EAR_TRAGION': {'x': 1050, 'y': 981, 'z': 339.314208984375},
  'LEFT_EYE_RIGHT_CORNER': {'x': 255, 'y': 908, 'z': 2.906157255172729},
  'MOUTH_CENTER': {'x': 514, 'y': 1470, 'z': -14.335670471191406},
  'RIGHT_EYEBROW_UPPER_MIDPOINT': {'x': 696,
   'y': 649,
   'z': -31.68943977355957},
  'RIGHT_EYE_BOTTOM_BOUNDARY': {'x': 710, 'y': 874, 'z': 7.907994270324707},
  'MOUTH_RIGHT': {'x': 722, 'y': 1417, 'z': 37.667259216308594},
  'LEFT_EAR_TRAGION': {'x': -184, 'y': 1164, 'z': 321.07354736328125},
  'NOSE_TIP': {'x': 481, 'y': 1206, 'z': -107.38621520996094},
  'LEFT_EYE_BOTTOM_BOUNDARY': {'x': 154, 'y': 956, 'z': -0.18508356809616003},
  'FOREHEAD_GLABELLA': {'x': 412, 'y': 758, 'z': -62.1227722167